In [4]:
import os
from dotenv import load_dotenv
from pyathena import connect
import schedule
import time
import logging

# Configuração do logging
logging.basicConfig(filename='update_tables.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Carrega variáveis de ambiente
load_dotenv()
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region_name = os.getenv("AWS_REGION")
s3_staging_dir = os.getenv("S3_STAGING_DIR")

# Conecta ao Athena
conn = connect(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    s3_staging_dir=s3_staging_dir,
    region_name=region_name
)

def executa_query(cursor, query, success_message, error_message):
    try:
        cursor.execute(query)
        logging.info(success_message)
    except Exception as e:
        logging.error(f"{error_message}: {e}")

def read_query(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def atualiza_tabelas():
    try:
        with conn.cursor() as cursor:
            # Passo 1: Excluir as tabelas existentes
            deleta_tabelas = [
                "DROP TABLE IF EXISTS prevencao_a_fraude.qtd_falta;",
                "DROP TABLE IF EXISTS prevencao_a_fraude.data_inventario;",
                "DROP TABLE IF EXISTS prevencao_a_fraude.vendas_cambraia;",
                "DROP TABLE IF EXISTS prevencao_a_fraude.filial;",
                "DROP TABLE IF EXISTS prevencao_a_fraude.usuario;",
                "DROP TABLE IF EXISTS prevencao_a_fraude.cupomfiscal;"
            ]
            for query in deleta_tabelas:
                executa_query(cursor, query, "Tabela excluída com sucesso.", "Erro ao excluir tabela")
            
            # Passo 2: Criar as tabelas novas
            criar_tabelas = [
                ('CONSULTAS/kf_data_inv.sql', "Tabela data_inventario criada com sucesso.", "Erro ao criar tabela data_inventario"),
                ('CONSULTAS/kf_qtd_falta.sql', "Tabela qtd_falta criada com sucesso.", "Erro ao criar tabela qtd_falta"),
                ('CONSULTAS/vendas.sql', "Tabela vendas_cambraia criada com sucesso.", "Erro ao criar tabela vendas_cambraia"),
                ('CONSULTAS/FILIAL.SQL', "Tabela FILIAL criada com sucesso.", "Erro ao criar tabela FILIAL"),
                ('CONSULTAS/USUARIO.SQL', "Tabela USUARIO criada com sucesso.", "Erro ao criar tabela USUARIO"),
                ('CONSULTAS/CupomFiscal.sql', "Tabela CupomFiscal criada com sucesso.", "Erro ao criar tabela CupomFiscal")
            ]
            for file_path, se_sucesso, se_erro in criar_tabelas:
                query = read_query(file_path)
                executa_query(cursor, query, se_sucesso, se_erro)

    except Exception as e:
        logging.error(f"Erro geral: {e}")
    finally:
        conn.close()

# Atualiza as tabelas diariamente
schedule.every().day.at("14:43").do(atualiza_tabelas)

while True:
    schedule.run_pending()
    time.sleep(1) 